In [79]:
import rasterio
from rasterio.transform import from_origin
from rasterio.enums import Resampling
from rasterio.features import geometry_mask
import geopandas as gpd
from shapely.geometry import mapping

In [80]:
def create_binary_image(sar_image_path, shapefile_path, output_image_path):
    # Read SAR image
    with rasterio.open(sar_image_path) as src:
        sar_data = src.read(1)
        transform = src.transform
        profile = src.profile

    # Read shapefile
    gdf = gpd.read_file(shapefile_path)
    mask = geometry_mask(gdf.geometry, out_shape=sar_data.shape, transform=transform, invert=True)

    # Create a new image with the same dimensions as the SAR image
    binary_image = mask.astype('uint8') * 255

    # Update the profile for the output image
    profile.update(count=1, dtype='uint8')

    # Write the new image
    with rasterio.open(output_image_path, 'w', **profile) as dst:
        dst.write(binary_image, 1)

In [81]:
if __name__ == "__main__":
    # for i in range(1,11):
    sar_image_path = f"data/subset_5_of_final_S1A_IW_GRDH_1SDV_20240323T094000_20240323T094025_053107_066EC8_11F5.tif"
    shapefile_path = f"./shapefiles/subset_5_of_final_S1A_IW_GRDH_1SDV_20240323T094000_20240323T094025_053107_066EC8_11F5.shp"
    output_image_path = f"./output_tif/subset_5_of_final_S1A_IW_GRDH_1SDV_20240323T094000_20240323T094025_053107_066EC8_11F5_new_mask.tif"

    create_binary_image(sar_image_path, shapefile_path, output_image_path)
        # print(f'processed image {i}')